# Noise Reduction with U-Net

In `01_preprocessing.ipynb` the images were normalized by resampling to a uniform size, clipping of very high intensities and removing field bias. Not only do these steps take up a lot of time (especially field bias removal), they also leave some noise and variation on the image data. This makes it hard to fit 3D CNNs on the data. Our aproach is therefore to use a U-Net to further presprocess the images. Here the original, not the already preprocessed images, are used as input and the preprocessed images are used as targets. 

In [ ]:
import SimpleITK as sitk
sitk.ProcessObject_SetGlobalDefaultNumberOfThreads(1)

In [ ]:
import sys
sys.path.append('..')

from fastai.callback.all import SaveModelCallback

import faimed3d
from faimed3d.all import *
from faimed3d.models.unet import _get_sz_change_idxs
from faimed3d.augment import _get_margins, _add_to_margin
from libs.utils import *
from libs.augment import *

## Construct DataLoader

In [ ]:
train = False
inference = True
sequence = 'T1'
model_type = 'resnet18_3d'
model_fun = getattr(faimed3d.all, model_type)
model_name = f'denoising_unet_{model_type}_{sequence}'

In [ ]:
df = get_dataframe(preprocessed_images=False)
df2 = get_dataframe()

In [ ]:
df['preprocessed_TIRM'] = df2.TIRM
df['preprocessed_T1'] = df2.T1
df['TIRM'] = [fn.replace('../data/', '') for fn in df.TIRM]
df['T1'] = [fn.replace('../data/', '') for fn in df.T1]
DATA_DIR = Path('/media/ScaleOut/vahldiek/MRI/SIJ/')

In [ ]:
@Transform
def ScaleAndUnsequeeze(x:(TensorDicom3D, TensorMask3D)):
    x = x.float()
    x = (x - x.mean()) / x.std()
    return x.unsqueeze(1)

In [ ]:
dblock = DataBlock(blocks = (ImageBlock3D, ImageBlock3D(cls=TensorMask3D)), 
                    getters = [lambda x: DATA_DIR/x[sequence], lambda x: DATA_DIR/x[f'preprocessed_{sequence}']], 
                    item_tfms = [Resample3D((20, 224, 224), (3.5, 1, 1)), *TioTransforms(0.3), Resize3D((20,192,192)), RandomNoise3D(p=0.25)], 
                    batch_tfms=  ScaleAndUnsequeeze,
                    splitter = ColSplitter('is_valid')
                   )
dls = dblock.dataloaders(df, bs = 1)

In [ ]:
def get_model(**kwargs): 
    model = model_fun(**kwargs)
    model.stem[0] = nn.Conv3d(3, 32, kernel_size=3, stride = 1, padding = 0)
    return model

In [ ]:
learn = unet_learner_3d(dls, 
                        get_model, 
                        model_dir = '/home/bressekk/Documents/deep-spa-mri/models/',
                        cbs=SaveModelCallback(fname=model_name, with_opt=True), 
                        loss_func=MSELossFlat(), 
                        n_out = 1).to_fp16()
learn.unfreeze()

In [ ]:
# cuda
if train: 
    learn.fit_one_cycle(5, 1e-3, wd = 1e-2)

In [ ]:
if train: 
    learn.recorder.plot_loss(skip_start=100)

In [ ]:
learn = learn.load(model_name)

The model can now be used to denoise images of above specified `sequence`. 